In [1]:
class Point:
    pass

Point is inhereting from object. Object implements default new and init method. 

`__new__` method creates new instance of a class

In [3]:
p = Point()

In [4]:
type(p)

__main__.Point

we can create a new instance of a Point another way

In [5]:
p = object.__new__(Point)

In [6]:
p, type(p)

(<__main__.Point at 0x252027b15c8>, __main__.Point)

Let's add init to Point

In [8]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [9]:
p = object.__new__(Point)

p is instance of Point, but hasn't run the init yet

In [10]:
p.__dict__

{}

In [11]:
p.__init__(10, 20)

In [12]:
p.__dict__

{'x': 10, 'y': 20}

`__new__` method allows to pass any number of arguments, but doesn't do anything with them

In [13]:
p = object.__new__(Point, 10, 20)

In [14]:
p.__dict__

{}

let's override `__new__`. it's static method, first parameter is a class to create. we don't need static decorator. python knows it's static

In [16]:
class Point:
    def __new__(cls, x, y):
        print("Creating instance...", x, y)
        instance = object.__new__(cls)
        return instance
    def __init__(self, x, y):
        print("init called...", x, y)
        self.x = x
        self.y = y

In [17]:
p = Point(10, 20)

Creating instance... 10 20
init called... 10 20


and we should have called super

In [18]:
class Point:
    def __new__(cls, x, y):
        print("Creating instance...", x, y)
        instance = super().__new__(cls)
        return instance
    def __init__(self, x, y):
        print("init called...", x, y)
        self.x = x
        self.y = y

In [19]:
p = Point(0, 2)

Creating instance... 0 2
init called... 0 2


#### Inheritance from built-in classes

In [20]:
class Squared(int):
    def __new__(cls, x):
        return super().__new__(cls, x**2)

In [21]:
result = Squared(4)
result

16

In [22]:
type(result), isinstance(result, int)

(__main__.Squared, True)

in init that would not work: cause built-ins are written in C and behave differently

In [23]:
class Squared(int):
    def __init__(self, x):
        print("calling init...")
        super().__init__(x**2)

In [24]:
result = Squared(4)

calling init...


TypeError: object.__init__() takes exactly one argument (the instance to initialize)

#### why we need to call super

In [25]:
class Person:
    def __new__(cls, name):
        print(f"Person instantiating {cls.__name__}...")
        instance = object.__new__(cls)
        return instance
    def __init__(self, name):
        print("Person: initializing instance")
        self.name = name

In [26]:
p = Person("Alex")

Person instantiating Person...
Person: initializing instance


In [27]:
p.__dict__

{'name': 'Alex'}

but it won't work with inheritance

In [30]:
class Student(Person):
    def __new__(cls, name, major):
        print(f"Student: instantiating {cls.__name__}...")
        instance = object.__new__(cls)
        return instance
    def __init__(self, name, major):
        print("Student: initializing instance")
        super().__init__(name)
        self.major = major

In [31]:
s= Student("Alex", "English") # __new__ method of Person wasn't called

Student: instantiating Student...
Student: initializing instance
Person: initializing instance


to insure that we call `__new__` from the parent class:

(in this case it actualy didn't matter, cause `__new__` doesn't do anything)

In [32]:
class Student(Person):
    def __new__(cls, name, major):
        print(f"Student: instantiating {cls.__name__}...")
        instance = super().__new__(cls, name)
        return instance
    def __init__(self, name, major):
        print("Student: initializing instance")
        super().__init__(name)
        self.major = major

In [33]:
s= Student("Alex", "English")

Student: instantiating Student...
Person instantiating Student...
Student: initializing instance
Person: initializing instance


#### how can we use `__new__`

In [44]:
class Square:
    def __new__(cls, w, l):
        cls.area = lambda self: self.w * self.l
        instance = super().__new__(cls)
        return instance
    def __init__(self, w, l):
        self.w = w
        self.l = l

In [45]:
s = Square(3, 4)

In [46]:
s.area

<bound method Square.__new__.<locals>.<lambda> of <__main__.Square object at 0x0000025202880588>>

In [47]:
s.area()

12

we can put everything in `__new__`

In [50]:
class Square:
    def __new__(cls, w, l):
        cls.area = lambda self: self.w * self.l
        instance = super().__new__(cls)
        instance.w = w
        instance.l = l
        return instance

In [51]:
s = Square(2, 4)

In [52]:
s.__dict__

{'w': 2, 'l': 4}

In [53]:
s.area()

8

we can invoke `__new__` ourselves. it won't call init, but we have all the initialization in the  `__new__` method. that's how built-ins work

In [54]:
s = Square.__new__(Square, 1, 9)

In [55]:
s.area()

9

if `__new__` doesn't return instance of created class, init won't get called

In [56]:
class Person:
    def __new__(cls, name):
        print(f"Person instantiating {cls.__name__}... not really ...")
        instance = str(name)
        return instance
    

In [57]:
p = Person("alex")

Person instantiating Person... not really ...


In [58]:
p, type(p)

('alex', str)

In [59]:
class Person:
    def __new__(cls, name):
        print(f"Person instantiating {cls.__name__}... not really ...")
        instance = str(name)
        return instance
    def __init__(self, name, ): # won't be called
        print("init called")
        self.name = name

In [60]:
p = Person("alex")

Person instantiating Person... not really ...
